In [1]:
import pandas as pd
import numpy as np

In [35]:
data = pd.read_csv('/home/robbe/multimodal_dataset/outputs/aligned_dataset.csv')

In [36]:
data

,Modified sequence,Intensity,Charge,m/z,Retention time,Raw file,Experiment,Accession,1/K0,CCS,Relative_Std_Dev_CCS,Conformer,Conformer id,K0,mods
0,_TTGIVMDSGDGVTHTVPIYEGYALPHAILR_,108210000.0,4,796.659035,80.276,21-Prot-1910_GA4_1_6288,Meso109F-hTERT-EGFP_NE2_1,PXD036127,1.111596,892.064183,6.704142,True,1.0,NaN,NaN
1,_TTGIVM(Oxidation (M))DSGDGVTHTVPIYEGYALPHAILR_,71644000.0,3,1067.207930,75.803,21-Prot-1902_BE4_1_6309,VMC59F_NE3_2,PXD036127,1.285679,775.697036,3.246908,True,1.0,NaN,Oxidation (M)
2,_IWHHTFYNELR_,61934000.0,3,505.921237,39.095,21-Prot-1898_BD8_1_6271,Meso125F-hTERT-EGFP_NE2_1,PXD036127,0.838385,513.247255,1.939613,False,1.0,NaN,NaN
3,_MSVQPTVSLGGFEITPPVVLR_,58098000.0,2,1114.111390,99.507,21-Prot-1907_GA1_1_6284,Meso109F_NE2_1,PXD036127,1.262304,510.776675,0.645627,False,1.0,NaN,NaN
4,_IWHHTFYNELR_,56156000.0,3,505.921237,38.707,21-Prot-1897_BD7_1_6303,Meso125F-hTERT-EGFP_NE1_2,PXD036127,0.838385,506.450034,1.638973,False,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9679924,_NKYEDEINKR_,6411.6,3,436.889603,10.399,201207_Ch1_RD7_1_3165,Ch1,PXD038824,0.750617,454.463557,0.230401,False,1.0,NaN,NaN
9679925,_YENEVALR_,6373.7,2,497.253631,15.606,210503_Ch2_RB4_1_3548,Ch2,PXD038824,0.854005,354.224311,0.000000,False,1.0,NaN,NaN
9679926,_TSKNDDSHTLEAEMSYR_,5212.3,4,496.723272,14.642,201207_Ch1_BE5_1_3075,Ch1,PXD038824,0.841000,697.098874,0.000000,False,1.0,NaN,NaN
9679927,_LSSPATLNSR_,5048.8,2,523.285463,15.103,210503_Ch1_20ul_RB6_1_3553,Ch1_20ul,PXD038824,0.826680,345.541553,0.405096,False,1.0,NaN,NaN


In [37]:
conformers_PXD = data.groupby(['Modified sequence', 'Charge', 'Accession', 'Conformer id'])[['CCS', 'Intensity']].mean().reset_index()

In [38]:
def check_modal_dict_vector(modal_dict, CCS):
    CCS_array = np.array(list(modal_dict.values()))
    mask = ~(
        (CCS > (CCS_array + 0.02 * CCS_array)) | (CCS < (CCS_array - 0.02 * CCS_array))
    )
    matching_keys = np.where(mask)[0]
    if len(matching_keys) > 0:
        key = list(modal_dict.keys())[matching_keys[0]]
        modal_dict[key] = np.mean([CCS, CCS_array[matching_keys[0]]])
        return key, modal_dict
    else:
        return None, modal_dict

def assign_conformer_ids(group):
    sorted_group = group.sort_values(["CCS", "Intensity"], ascending=[True, False])
    prev_CCS = 0
    conformer_id = 0
    conformer_ids = []
    modal_dict = {}
    for _, row in sorted_group.iterrows():
        id, modal_dict = check_modal_dict_vector(modal_dict, row["CCS"])
        if id is None:
            if not row["CCS"] > (prev_CCS + 0.02 * prev_CCS):
                conformer_ids.append(conformer_id)
            else:
                conformer_id += 1
                modal_dict[conformer_id] = row["CCS"]
                conformer_ids.append(conformer_id)
                prev_CCS = row["CCS"]
        else:
            conformer_ids.append(id)
    sorted_group["Conformer id total"] = conformer_ids
    return sorted_group

def get_multimodals(data):
    # PXD["Relative_Std_Dev_CCS"] = PXD.groupby(
    #     ["Modified sequence", "Charge", "Raw file"]
    # )["CCS"].transform(lambda x: np.std(x) / np.mean(x) * 100)
    # PXD["Conformer"] = False
    # conformers = PXD[PXD["Relative_Std_Dev_CCS"] > 2]
    # PXD.loc[conformers.index, "Conformer"] = True
    data.sort_values("Intensity", ascending=False, inplace=True)
    data_grouped = data.groupby(["Modified sequence", "Charge"])
    for _, group in data_grouped:
        sorted_group = assign_conformer_ids(group)
        data.loc[sorted_group.index, "Conformer id total"] = sorted_group["Conformer id total"]
        original_conformer_ids = np.sort(sorted_group["Conformer id total"].unique())
        id_counts = (
            sorted_group["Conformer id total"].value_counts().sort_values(ascending=False)
        )
        id_map = {
            original_conformer_ids: rank
            for rank, original_conformer_ids in enumerate(id_counts.index, start=1)
        }
        data.loc[sorted_group.index, "Conformer id total"] = sorted_group["Conformer id total"].map(
            id_map
        )
    return data

In [39]:
total_conformers = get_multimodals(conformers_PXD)

In [40]:
# Filter out conformers that don't occur in more than 1 Accession
grouped = total_conformers.groupby(['Modified sequence', 'Charge', 'Conformer id total'])['Accession'].nunique()
filtered_groups = grouped[grouped > 1]
final_conformers = total_conformers[total_conformers.set_index(['Modified sequence', 'Charge', 'Conformer id total']).index.isin(filtered_groups.index)]

In [42]:
final_conformers['Conformer id total'].value_counts()

Conformer id total
1.0    630239
2.0     28777
3.0      1782
4.0       217
5.0        47
6.0        14
7.0         2
Name: count, dtype: int64

In [43]:
final_conformers.to_csv('/home/robbe/multimodal_dataset/outputs/final_conformers.csv', index=False)

In [2]:
final_conformers = pd.read_csv('/home/robbe/multimodal_dataset/outputs/final_conformers.csv')

In [3]:
final_conformers_filtered = final_conformers[final_conformers['Conformer id total'].isin([1, 2])]

In [4]:
final_conformers_filtered.to_csv('/home/robbe/multimodal_dataset/outputs/final_conformers_filtered.csv', index=False)

In [5]:
conformers_unique = final_conformers_filtered.groupby(['Modified sequence', 'Charge', 'Conformer id total'])['CCS'].mean().reset_index()

In [6]:
conformers_unique.value_counts('Conformer id total')

Conformer id total
1.0    168306
2.0      9814
Name: count, dtype: int64

In [7]:
conformers_unique.to_csv('/home/robbe/multimodal_dataset/outputs/final_conformers_unique.csv', index=False)

In [8]:
# Save as pickle
conformers_unique.to_pickle('/home/robbe/multimodal_dataset/outputs/final_conformers_unique.pkl')

In [9]:
grouped = conformers_unique.groupby(['Modified sequence', 'Charge'])
grouped = grouped.apply(lambda x: x['CCS'].tolist())
grouped = grouped.reset_index()
grouped.columns = ['Modified sequence', 'Charge', 'CCS']

/tmp/ipykernel_3583573/1015950563.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = grouped.apply(lambda x: x['CCS'].tolist())


In [10]:
grouped['CCS'] = grouped['CCS'].apply(lambda x: x if len(x) == 2 else [x[0], x[0]])

In [11]:
grouped.to_csv('/home/robbe/multimodal_dataset/outputs/final_conformers_unique_grouped.csv', index=False)

In [12]:
# Save as pickle
conformers_unique.to_pickle('/home/robbe/multimodal_dataset/outputs/final_conformers_unique.pkl')

In [17]:
grouped.to_pickle('/home/robbe/multimodal_dataset/outputs/final_conformers_unique_grouped.pkl')

In [13]:
# Filter out peptides with only 1 conformer
only_multimodals = grouped[grouped['CCS'].apply(lambda x: x[0] != x[1])]

In [14]:
only_multimodals

,Modified sequence,Charge,CCS
44,_(Acetyl (Protein N-term))AAAAVSESWPELELAER_,2,"[505.99545184485044, 491.76811706732934]"
50,_(Acetyl (Protein N-term))AAAEAANCIMEVSCGQAESS...,3,"[670.4411322249787, 687.1896511507]"
62,_(Acetyl (Protein N-term))AAAPPSYCFVAFPPR_,2,"[459.03307469033274, 469.8057961334292]"
82,_(Acetyl (Protein N-term))AAEAADLGLGAAVPVELR_,2,"[476.35432039471, 504.9210970593681]"
104,_(Acetyl (Protein N-term))AAGTLYTYPENWR_,2,"[452.16576676429, 396.7513191105102]"
...,...,...,...
168131,_YYEAADTVTQFDNVR_,2,"[448.779553922157, 488.477357219641]"
168148,_YYEVLGAAATTDYNNNHEGREEDQR_,3,"[701.2994217947162, 681.1926815660267]"
168201,_YYKTIDDLKNQILNLTTDNANILLQIDNAR_,4,"[776.0356081115502, 700.4284038538108]"
168250,_YYPTEDVPR_,2,"[381.9621519913044, 359.97193294486567]"


In [15]:
only_multimodals.to_csv('/home/robbe/multimodal_dataset/outputs/only_multimodals.csv', index=False)

In [16]:
# Save as pickle
only_multimodals.to_pickle('/home/robbe/multimodal_dataset/outputs/only_multimodals.pkl')